# LLM Converstion
Conversation between three LLMs 
1. Gemini (gemini-2.5-pro)
2. OpenAPI (gpt-4.1-mini) 
3. nthropic (claude-sonnet-4-5-20250929) 

User can provide the following for each LLM:
1. Conversation Topic-Starter 
2. Personality type: Skeptic, Narcissist, Pessimist etc.
3. Gender: Male, Female, Neutral
4. Conversation Length: 1 to 10. Default 5 (How many time each LLM responses)

OpenAPI client library is used for all the LLMs as they have OpenAI compatible end points. LLM specific libraries are not used.

In [61]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import base64
from io import BytesIO
from PIL import Image

In [62]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")



OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Anthropic API Key exists and begins sk-ant-


In [63]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

# For Gemini and Groq, we can use the OpenAI python client
# Because they have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
anthropic_url = "https://api.anthropic.com/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

openai_model = "gpt-4.1-mini"
gemini_model = "gemini-2.5-pro"
anthropic_model='claude-sonnet-4-5-20250929'

openai_llm = "OpenAI"
gemini_llm = "Gemini"
anthropic_llm = "Anthropic"

In [64]:
PERSONALITY_MAP = {
    "Skeptic": "Critical and Questioning (seeking flaws and evidence)",
    "Narcissist": "Arrogant and Self-Aggrandizing (focused entirely on self-superiority)",
    "Pessimist": "Gloomy and Fatalistic (focused on inevitable failure and doom)",
    "Optimist": "Enthusiastic and Encouraging (focused on positive possibility and success)",
    "Joker": "Wry and Comedic (using humor, sarcasm, or light absurdity)",
    "Stoic": "Detached and Analytical (emotionless, logical, and objective)",
    "Philosopher": "Contemplative and Existential (pondering deep meaning and abstract concepts)",
    "Bureaucrat": "Formal and Procedural (focused on rules, documents, and official terminology)",
    "Gossip": "Intimate and Distractible (focused on personal details, rumors, and asides)",
    "Mentor": "Didactic and Authoritative (teaching, offering advice, and guiding instruction)"
}

# Example of how you would use it:
# selected_personality = "Narcissist"
# print(PERSONALITY_MAP[selected_personality])
# Output: Believes they are superior, redirects topics to their own achievements, and dismisses others.

In [65]:
def generate_system_prompt (llm_type, personality_type):
    """
    Constructs the detailed system instruction, implementing defaults for invalid inputs.
    Uses 'Neutral' for gender and 'Skeptic' for personality if not provided or invalid.
    """
    
    # --- 1. Define Defaults and Apply Fallbacks ---
    default_personality = "Skeptic"

    # Check and set personality type, falling back to default if invalid
    if personality_type in PERSONALITY_MAP:
        final_personality = personality_type
    else:
        final_personality = default_personality
        print(f"Warning: Invalid personality '{personality_type}' for {llm_type}. Defaulting to '{default_personality}'.")


    # --- 2. Construct Prompt Components ---

    # Base behavior description from the map
    behavior_description = PERSONALITY_MAP[final_personality]


    # --- 3. Combine Instructions ---
    
    final_prompt = (
        f"You are the AI model based on the '{llm_type}' architecture. "
        f"Your primary goal is to interact in a multi-agent conversation. "
        f"Your **primary personality instruction** is to act as a **{final_personality}**. "
        f"Specifically: {behavior_description} "
        f"You are in a 3-way conversation with two other AI models. "
        "Keep your responses concise (1-3 sentences maximum) and strictly adhere to your assigned persona and gender. "
        "**Crucially, do not announce your name or persona.** Just provide your response directly."
    )
    
    return final_prompt


In [66]:
def generate_user_prompt(conversation_sofar):
    return f"""
    The conversation so far is as follows:
    {conversation_sofar}
    Now with this, respond with what you would like to say next.
    """

In [67]:
def call_llm(llm, model, system_prompt, user_prompt):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    response = llm.chat.completions.create(model=model, messages=messages) # type: ignore
    return response.choices[0].message.content

In [68]:
'''
print("OpenAI")
print (call_llm(openai, openai_model, "You are tech assistant", "How tcpip works"))

print("Gemini")
print (call_llm(gemini, gemini_model, "You are tech assistant", "How tcpip works"))

print("Anthropic")
print (call_llm(anthropic, anthropic_model, "You are tech assistant", "How tcpip works"))
'''

'\nprint("OpenAI")\nprint (call_llm(openai, openai_model, "You are tech assistant", "How tcpip works"))\n\nprint("Gemini")\nprint (call_llm(gemini, gemini_model, "You are tech assistant", "How tcpip works"))\n\nprint("Anthropic")\nprint (call_llm(anthropic, anthropic_model, "You are tech assistant", "How tcpip works"))\n'

In [69]:
def conversation(openai_personality, gemini_personality, anthropic_personality,conversation_starter, conversation_length):

    import time

    openai_system_prompt = generate_system_prompt(openai_llm, openai_personality)
    gemini_system_prompt = generate_system_prompt(gemini_llm, gemini_personality)
    anthropic_system_prompt = generate_system_prompt(anthropic_llm, anthropic_personality)

    
    conversation_sofar = f"""
- **{openai_llm}**: {openai_personality} -- {PERSONALITY_MAP[openai_personality]}
- **{gemini_llm}**: {gemini_personality} -- {PERSONALITY_MAP[gemini_personality]}
- **{anthropic_llm}**: {anthropic_personality} -- {PERSONALITY_MAP[anthropic_personality]}

**Topic/Starter:** {conversation_starter}
"""

    yield conversation_sofar

    try:
        conversation_length = int(conversation_length)
    except (ValueError, TypeError):
        conversation_length = 3 # Default value

    sleep_interval = 0.0
    for i in range(int(conversation_length)):
        #round_header = f"\n\n---\n### Round {i+1}\n---\n"
        round_header = "\n\n"
        conversation_sofar += round_header
        yield conversation_sofar
        time.sleep(sleep_interval)
        
        # OpenAI
        conversation_sofar += "\n**🤖 OpenAI:** " + call_llm(openai, openai_model, openai_system_prompt, generate_user_prompt(conversation_sofar)) # type: ignore
        yield conversation_sofar
        time.sleep(sleep_interval)
        
        # Gemini
        conversation_sofar += "\n\n**✨ Gemini:** " + call_llm(gemini, gemini_model, gemini_system_prompt, generate_user_prompt(conversation_sofar)) # type: ignore
        yield conversation_sofar
        time.sleep(sleep_interval)

        # Anthropic
        conversation_sofar += "\n\n**🧠 Anthropic:** " + call_llm(anthropic, anthropic_model, anthropic_system_prompt, generate_user_prompt(conversation_sofar)) # type: ignore
        yield conversation_sofar
    conversation_sofar += "\n\n**---      End of debate     ---**\n"
    yield conversation_sofar

In [70]:
conversation("Stoic", "Skeptic", "Joker", "What is the purpose of life", 5);

In [ ]:
def getImage(conversation_text):
    """
    Feed the entire conversation text to an LLM.
    The LLM extracts: topic, personalities, mood, etc.
    It generates a clean DALL·E prompt.
    Then we call the image generator.
    """

    # Step 1: Ask LLM to convert conversation → image prompt
    analysis_response = openai.chat.completions.create(
        model="gpt-4.1",  # or "gpt-4.1-mini" etc.
        messages=[
            {
                "role": "system",
                "content": (
                    "Your job is to read a conversation between multiple AI personalities "
                    "and generate a single, unified image prompt for a DALL·E-style model. "
                    "Rules:\n"
                    "- DO NOT include dialogue or text from the conversation.\n"
                    "- Infer the personalities from tone, behavior, and labels.\n"
                    "- Infer the topic, even if the formatting varies.\n"
                    "- Create a symbolic, metaphorical scene representing the debate.\n"
                    "- No speech bubbles or text in the image.\n"
                    "- The prompt should describe one cohesive illustration.\n"
                    "- Image should contain the topic text"
                    "Output ONLY the final image prompt with no explanation."
                )
            },
            {
                "role": "user",
                "content": (
                    "Here is the full conversation. "
                    "Please generate the best possible image prompt:\n\n"
                    f"{conversation_text}"
                )
            }
        ],
        temperature=0.4
    )

    dalle_prompt = analysis_response.choices[0].message.content.strip() # type: ignore
    print(dalle_prompt)

    # Step 2: Generate the image using the LLM-generated prompt
    image_response = openai.images.generate(
        model="dall-e-3",    
        prompt=dalle_prompt,
        size="1024x1024",
        response_format="b64_json"
    )

    image_base64 = image_response.data[0].b64_json # type: ignore
    image_data = base64.b64decode(image_base64) # type: ignore
    return Image.open(BytesIO(image_data))




In [ ]:
import gradio as gr
import numpy as np
from scipy.io.wavfile import write
import matplotlib.pyplot as plt

def getAudio(text):
    # Dummy function: returns a path to a generated audio file
    print("Generating audio for text...")
    samplerate = 44100
    fs = 100
    t = np.linspace(0., 1., samplerate)
    amplitude = np.iinfo(np.int16).max
    data = amplitude * np.sin(2. * np.pi * fs * t)
    audio_path = "dummy_audio.wav"
    write(audio_path, samplerate, data.astype(np.int16))
    return audio_path

''' 
def getImageOld(text):
    # Dummy function: returns a path to a generated image file
    print("Generating image for text...")
    plt.figure()
    plt.plot([0, 1], [0, 1]) # simple plot
    plt.title("Conversation Visualization")
    image_path = "dummy_image.png"
    plt.savefig(image_path)
    plt.close()
    return image_path
'''

def get_media(text, generate_audio, generate_image):
    audio_path = None
    image_path = None
    if generate_audio:
        audio_path = getAudio(text)
    if generate_image:
        image_path = getImage(text)
    return audio_path, image_path

def update_visibility(checked):
    return gr.update(visible=checked)

# --- Gradio UI ---
# Reformat the personality map to be used in the dropdown choices
# The format is a list of tuples: (display_name, internal_value)
personality_choices = [
    (f"{key}: {desc}", key) 
    for key, desc in PERSONALITY_MAP.items()
]

with gr.Blocks() as demo:
    gr.Markdown("""# 🤖 LLM Round Table Conversation
    Set the personalities for three LLMs (OpenAI, Gemini, Anthropic) and watch them discuss a topic. The conversation is streamed turn-by-turn.""")
    with gr.Row():
        openai_personality = gr.Dropdown(personality_choices, label="OpenAI Personality", value="Stoic")
        gemini_personality = gr.Dropdown(personality_choices, label="Gemini Personality", value="Skeptic")
        anthropic_personality = gr.Dropdown(personality_choices, label="Anthropic Personality", value="Joker")
    conversation_starter = gr.Textbox(label="Conversation Starter", lines=2, placeholder="e.g., What is the purpose of life?")
    conversation_length = gr.Slider(1, 10, step=1, label="Conversation Length (Rounds)", value=1)
    
    with gr.Row():
        audio_checkbox = gr.Checkbox(label="Generate Audio", value=False)
        image_checkbox = gr.Checkbox(label="Generate Image", value=False)

    start_button = gr.Button("Start Conversation")
    
    output_markdown = gr.Markdown(label="Live Conversation")
    
    output_audio = gr.Audio(label="Conversation Audio", visible=False)
    output_image = gr.Image(label="Conversation Image", visible=False)

    audio_checkbox.change(fn=update_visibility, inputs=audio_checkbox, outputs=output_audio)
    image_checkbox.change(fn=update_visibility, inputs=image_checkbox, outputs=output_image)

    start_button.click(
        fn=conversation,
        inputs=[
            openai_personality, 
            gemini_personality, 
            anthropic_personality,
            conversation_starter, 
            conversation_length
        ],
        outputs=output_markdown
    ).then(
        fn=get_media,
        inputs=[output_markdown, audio_checkbox, image_checkbox],
        outputs=[output_audio, output_image]
    )

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


A surreal courtroom scene where three distinct figures preside over a single, glowing seedling emerging from a crack in a stone floor. One figure is a stoic, marble statue holding scientific instruments, another is a skeptical detective with a magnifying glass, and the third is a jester balancing on a unicycle, juggling question marks. The background is a cosmic expanse, blending logic, inquiry, and humor, symbolizing the debate about the definition of life.
A whimsical park at twilight: on one side, a shadowy figure sits under a raincloud, surrounded by deflated balloons and wilting flowers; in the center, a cheerful character leaps onto a vibrant carousel, scattering confetti and chasing butterflies; on the other side, a stern official in a suit stands at a kiosk, stamping paperwork while holding a clipboard labeled with fun-related forms. The scene is divided by subtle lines, symbolizing contrasting attitudes toward fun, with playful and bureaucratic motifs blending at the borders.
